In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
np.random.seed(30)

In [ ]:
def parse_training_data(path):
    with open(path, 'r') as file:
        line = file.readline().strip()
        line_counter = 0
        while line:
            line_counter += 1

            if line_counter == 2: # the line contains the values of P, N, M
                line = line[1:] # skip the # sign at the beginning of the line
                line_segments = line.split(' ')
                for segment in line_segments:
                    segment = segment.strip()
                    if segment != '':
                        if segment[0] == 'P': # P=<P_VALUE>
                            P = int(segment[2:])
                        elif segment[0] == 'N': # N=<N_VALUE>
                            N = int(segment[2:])
                        elif segment[0] == 'M': # M=<M_VALUE>
                            M = int(segment[2:])
                
                X_train = np.empty((P, N))
                Y_train = np.empty((P, M))
            
            elif line_counter >= 3: # contains training data as well as desired output starting at line 3
                # line_segments would be sth. like:
                # ['1.0', '1.0', '1.0', '0.9']
                # where the first N items are the values
                # for input data dimensions and the
                # last M items are the last layer's
                # outputs

                line_segments = line.split(' ')
                line_segments = list(filter(lambda seg: seg != '', line_segments))
                X_train[line_counter-3] = np.asarray(list(map(float, line_segments[:N])))
                Y_train[line_counter-3] = np.asarray(list(map(float, line_segments[-M:])))
                
            line = file.readline().strip()
    
    return P, N, M, X_train, Y_train

The following function calculated the output of an RBF neuron given the input vector (also called stimulus) $x$, neuron's center $c$, and the width of the Gaussian function $s$ as follows:

$$output = e^{-\frac{||x-c||_2}{2\cdot s^2}}$$

In [ ]:
def radial_transferfunction(x, c, s):
    '''
    Given a pattern, center, and s value, 
    the function calculates the output of 
    the rbf neuron.

    Parameters
    ----------
    x : input vector (stimulus)
    c : center of the neuron
    s : size/width of the neuron
    '''
    
    e = math.e 
    distance = np.linalg.norm(x-c)
    pw = -1 * (distance / (2*(s**2))) 
    f = e**pw
    
    return f

In [ ]:
def calculate_output(x, C, weights, s):
    '''
    Given a pattern, Center and S values, 
    the function calculates the output of 
    the output layer neurons.  

    Parameters
    ----------
    x : input vector (stimulus)
    C : Centers of the neurons
    weights : weights of the neurons
              connecting the RBF layer 
              to the output layer
    s : size/width of the neurons
        
    '''
    output_dim = weights.shape[1]
    result = []
    out_m = np.empty((output_dim))
    sm = 0
    
    
    # calculate the output of the RBF layer
    for i in range(len(C)): # iterate over the centers in the RBF layer
        f = radial_transferfunction(x, C[i], s[i])
        result.append(f)
        
        
    for m in range(output_dim):
        for k in range(len(C)): # iterate over the centers in the RBF layer
            sm = sm + result[k]*weights[k][m]
        out_m[m] = sm
        sm = 0
    
     
    return result, out_m

In [ ]:
def update_weight(weights, y_train, predictions, p, result, eta):
    '''
    Given the weights, teacher values (ground truth), 
    output of the network and the pattern, this function
    updates the weights based on the following formula: 
    delta_weight = learning rate*(teacher - output)*result

    Parameters
    ----------
    weights : weights of the neurons
              connecting the RBF layer 
              to the output layer
    y_train : teacher values (ground truth)
    predictions : output of the network
    p : pattern number
    result : output of the RBF layer
    eta : learning rate
    '''
    
    output_dim = weights.shape[1]
    result_size = len(result)
    for m in range(output_dim):
        for k in range(result_size):
            delta_weight = eta * (y_train[p][m] - predictions[p][m]) * result[k]
            weights[k][m] = weights[k][m] + delta_weight
    
    return weights

In [ ]:
def MSE(P, Y_train, predictions):
    '''
    Given the patterns, teacher and output values, 
    the function calculates the mean squared error
    (MSE) for each output neuron

    Parameters
    ----------
    P : number of patterns
    Y_train : teacher values (ground truth)
    predictions : output of the network
    '''
    
    mean_squared_error = 0
    for p in range(P): # iterate over all samples
        mean_squared_error += ((Y_train[p]-predictions[p])**2) / P
    
    return mean_squared_error

In [ ]:
path = "./data/PA-B_training_data_03.txt"
P, input_dim, output_dim, X_train, Y_train = parse_training_data(path)

In [ ]:
eta = 0.001 # learning rate
EPOCHS = 10000  # Number of iterations

In [ ]:
# number of RBF neurons, taking it 
# less than the number of patterns
K = min(3, P)   


In [ ]:
# choosing center values Ck as a 
# subset of the input patterns
Ck = X_train[np.random.choice(X_train.shape[0], K, replace=False)]

In [ ]:
# no tips are given for our choice of 
# K and P, so we are assigning values 
# to sk randomly in the range [0, 1]
sk = np.random.random(K)

In [ ]:
# weights are initialized randomly
# in the range [-0.5, 0.5]
weights = np.random.random((K, output_dim)) # shape: (prev_neurons, cur_neurons) 0 <=weights <=1
weights -= 0.5 # -0.5 <= weights <= 0.5

In [ ]:
mean_squared_errors_list = [] # used to track of MSE in each iteration for further plotting

for epoch in range(EPOCHS):
    predictions = np.empty((P, output_dim)) # store the output of the model for whole samples, so this can be used further to evaluate the model
    for p in range(P): # iterate over training samples

        result, out_m = calculate_output(X_train[p], Ck, weights, sk)
        predictions[p] = out_m
        
        new_weights = update_weight(weights, Y_train, predictions, p, result, eta)
        weights = new_weights
        

    mean_squared_error = MSE(P, Y_train, predictions)
    mean_squared_error_average = np.mean(mean_squared_error) # this is used to find the avg error when the output is higher that 1 dim
    mean_squared_errors_list.append(mean_squared_error_average)

    if epoch % int(EPOCHS/10) == 0:
        print(f'Epoch:{epoch}, MSE={mean_squared_error}, avg: {mean_squared_error_average:.4f}')

